<a href="https://colab.research.google.com/github/MaJu502/proyectoDL/blob/main/Resnet_pytorch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto Deep Learning
- Diego Cordova 20212
- Marco Jurado 20308
- Cristian Aguirre 20231
- Paola Contreras 20213
- Paola de Leon 20361

## Carga de Dataset

In [5]:
!gdown 1BoNWFSXxtN74PgLcBxfGWyeKUOfevQEP

Downloading...
From: https://drive.google.com/uc?id=1BoNWFSXxtN74PgLcBxfGWyeKUOfevQEP
To: /content/kaggle.json
100% 64.0/64.0 [00:00<00:00, 365kB/s]


In [6]:
!gdown

usage: gdown [-h] [-V] [-O OUTPUT] [-q] [--fuzzy] [--id] [--proxy PROXY] [--speed SPEED]
             [--no-cookies] [--no-check-certificate] [--continue] [--folder] [--remaining-ok]
             url_or_id
gdown: error: the following arguments are required: url_or_id


In [7]:
! pip install kaggle

In [8]:
!ls -a /root/

.   .bashrc  .config   .jupyter  .keras		.local	.nv	  .tmux.conf
..  .cache   .ipython  .kaggle	 .launchpadlib	.npm	.profile  .wget-hsts


In [9]:
!mkdir /root/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [10]:
!mv /content/kaggle.json /root/.kaggle/


In [11]:
ls /root/.kaggle/


kaggle.json


In [12]:
!chmod 600 /root/.kaggle/kaggle.json

In [13]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

face-expression-recognition-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [14]:
%cd /content
!unzip -q face-expression-recognition-dataset.zip

/content
replace images/images/train/angry/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Modelo

In [15]:
import os
import shutil

# Ruta de la carpeta que deseas eliminar
folder_path = './images/train/disgust'

# Elimina todas las imágenes en la carpeta
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'Error al eliminar {file_path}: {e}')

# Elimina la carpeta misma
try:
    os.rmdir(folder_path)
except Exception as e:
    print(f'Error al eliminar la carpeta {folder_path}: {e}')


FileNotFoundError: ignored

In [ ]:
# Ruta de la carpeta que deseas eliminar
folder_path = './images/validation/disgust'

# Elimina todas las imágenes en la carpeta
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'Error al eliminar {file_path}: {e}')

# Elimina la carpeta misma
try:
    os.rmdir(folder_path)
except Exception as e:
    print(f'Error al eliminar la carpeta {folder_path}: {e}')

In [16]:
import os
import random
import shutil

# Ruta de la carpeta que deseas modificar
folder_path = './images/train/happy'

# Lista de archivos en la carpeta
file_list = os.listdir(folder_path)

# Número de imágenes que deseas eliminar
num_images_to_delete = 3000

# Verifica si hay suficientes imágenes para eliminar
if len(file_list) < num_images_to_delete:
    print(f'No hay suficientes imágenes en {folder_path} para eliminar.')
else:
    # Selecciona aleatoriamente las imágenes que se eliminarán
    images_to_delete = random.sample(file_list, num_images_to_delete)

    # Elimina las imágenes seleccionadas
    for file_name in images_to_delete:
        file_path = os.path.join(folder_path, file_name)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Error al eliminar {file_path}: {e}')

    print(f'Se han eliminado {num_images_to_delete} imágenes de {folder_path}.')


Se han eliminado 3000 imágenes de ./images/train/happy.


In [ ]:
import cv2
import os
import numpy as np

root_dir = './images/train/'

# Lista de emociones
emotions = ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprise']
unique_shapes_by_emotion = {emotion: [] for emotion in emotions}

for emotion in emotions:
    emotion_dir = os.path.join(root_dir, emotion)
    image_files = os.listdir(emotion_dir)

    for image_file in image_files:
        image_path = os.path.join(emotion_dir, image_file)
        image = cv2.imread(image_path)
        shape = image.shape
        if shape not in unique_shapes_by_emotion[emotion]:
            unique_shapes_by_emotion[emotion].append(shape)

for emotion, shapes in unique_shapes_by_emotion.items():
    print(f"Shapes únicos para la emoción '{emotion}':")
    for shape in shapes:
        print(shape)


## Loader para el Modelo

In [ ]:
import torch
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision import transforms as T

batch_size = 128
base_path = "./images/"

train_augs = T.Compose([T.RandomHorizontalFlip(p = 0.5),T.RandomRotation(degrees = (-20,+20)),T.ToTensor()])
#To tensor in pytorch converts image from numpy or PIL into pytorch tensors also it will convert (h,w,c)->(c,h,w)
valid_augs = T.Compose([T.ToTensor()])


# Crear conjuntos de datos
train_dataset = ImageFolder(base_path + "train", transform =train_augs)
validation_dataset = ImageFolder(base_path + "validation", transform =valid_augs)

# Crear generadores de lotes (DataLoaders)
trainloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle = True)
validloader = torch.utils.data.DataLoader(validation_dataset,batch_size=batch_size)

## Se preparan los datos para CUDA

## Entreno del Modelo

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install timm

In [ ]:
import timm
from torch import nn

class FaceModel(nn.Module):
    def __init__(self):
        super(FaceModel,self).__init__()
        self.eff_net = timm.create_model('resnet34',pretrained = True,num_classes = 7)
    def forward(self,images,labels = None):
        logits = self.eff_net(images)
        if labels != None:
            loss = nn.CrossEntropyLoss()(logits,labels)
            return logits,loss
        return logits

modelo = FaceModel()

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}\n")
modelo = modelo.to(device)

device_in_model = 'cpu' if False in [i.is_cuda for i in modelo.parameters()] else 'cuda'
print('model using', device_in_model)

In [ ]:
def multiclass_accuracy(y_pred,y_true):
    top_p,top_class = y_pred.topk(1,dim = 1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))

In [ ]:
from tqdm import tqdm

In [ ]:
epoch = 60


def train_fn(model,dataloader,optimizer,current_epo):
    model.train()
    total_loss = 0.0
    total_acc = 0.0
    tk = tqdm(dataloader,desc = "EPOCHS" + "[TRAIN]" + str(current_epo+1) + "/"+ str(epoch))
    for t,data in enumerate(tk):
        images,labels = data
        images,labels = images.to(device),labels.to(device)

        optimizer.zero_grad()
        logits,loss = model(images,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_acc += multiclass_accuracy(logits,labels)
        tk.set_postfix({'loss':'%6f' %float(total_loss/(t+1)),'acc':'%6f' %float(total_acc/(t+1))})
    return total_loss/len(dataloader),total_acc/len(dataloader)



In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

def eval_fn(model, dataloader, current_epo):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    all_true_labels = []
    all_predicted_labels = []

    tk = tqdm(dataloader, desc="EPOCH" + "[VALID]" + str(current_epo + 1) + "/" + str(epoch))

    for t, data in enumerate(tk):
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        logits, loss = model(images, labels)

        total_loss += loss.item()
        total_acc += multiclass_accuracy(logits, labels)

        # Collect true labels and predicted labels
        all_true_labels.extend(labels.cpu().numpy())
        all_predicted_labels.extend(logits.argmax(1).cpu().numpy())

        tk.set_postfix({'loss': '%6f' % float(total_loss / (t + 1)), 'acc': '%6f' % float(total_acc / (t + 1))})

    # Calculate and display the confusion matrix
    confusion = confusion_matrix(all_true_labels, all_predicted_labels)
    sns.heatmap(confusion, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.show()

    return total_loss / len(dataloader), total_acc / len(dataloader)




In [ ]:
optimizer = torch.optim.Adam(modelo.parameters(),lr = 0.0001)

In [ ]:
# best_valid_loss= np.inf
# for i in range(epoch):
#     train_loss,train_acc = train_fn(modelo,trainloader,optimizer,i)
#     eval_loss,eval_acc = eval_fn(modelo,validloader,i)
#     if eval_loss < best_valid_loss:
#         torch.save(modelo.state_dict(),'best-weights.pt')
#         print("Saved Best Valid Loss")
#         best_valid_loss = eval_loss

In [ ]:
# import joblib

# joblib.dump(modelo, 'modelo.pkl')

In [ ]:
import joblib
modelo_cargado = joblib.load('modelo.pkl')

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Paso 1: Cargar y preprocesar la imagen
image_path = "./images/validation/sad/10004.jpg"
image = Image.open(image_path).convert("RGB")

# Añadir transformaciones necesarias, incluyendo normalización
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

image = transform(image)

# Ajusta la dimensión del lote (batch dimension) si es necesario
image = image.unsqueeze(0)

# Paso 2: Mover el modelo a la CPU y realizar la predicción
modelo.to('cpu')  # Mover el modelo a la CPU

output = modelo(image)

# Para obtener las probabilidades de clase en un problema de clasificación
probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Paso 3: Interpretar los resultados
# Puedes obtener la clase predicha como el índice con mayor probabilidad
predicted_class = torch.argmax(probabilities).item()
print(f"La imagen probablemente pertenece a la clase: {predicted_class}")


## Mount persons folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p '/content/persons/'
!cp /content/drive/MyDrive/persons/persons.zip /content/persons/

In [ ]:
!unzip /content/drive/MyDrive/persons/persons.zip -d /content/persons/

In [ ]:
!rm /content/persons/persons.zip

## Predict video frames

In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

folder_path = "./persons/content/persons/"

transform = transforms.Compose([
    transforms.ToTensor()
])

class_labels = ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprise']

predictions = []
probabilities = []

for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path).convert("RGB")
        image = transform(image)
        image = image.unsqueeze(0)

        # Realizar la predicción
        output = modelo(image)
        probabilities_batch = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities_batch).item()

        predictions.append(predicted_class)
        probabilities.append(probabilities_batch)

# Convertir las listas en tensores
predictions = torch.tensor(predictions)
probabilities = torch.stack(probabilities)

# Calcular la distribución de las clases
class_distribution = torch.bincount(predictions, minlength=len(class_labels))

# Graficar
plt.figure(figsize=(10, 5))
plt.bar(class_labels, class_distribution)
plt.xlabel('Clases')
plt.ylabel('Número de Imágenes')
plt.title('Distribución de Clases')
plt.xticks(rotation=45)
plt.show()


In [ ]:
## Código para mover archivos a carpeta

# import os
# import shutil

# source_folder = './'

# destination_folder = 'persons/'

# if not os.path.exists(destination_folder):
#     os.makedirs(destination_folder)

# files = os.listdir(source_folder)

# for file in files:
#     if 'person_' in file:
#         source_path = os.path.join(source_folder, file)
#         destination_path = os.path.join(destination_folder, file)
#         shutil.move(source_path, destination_path)


In [ ]:
# ## Código para eliminar archivos en carpeta

# import os

# directory_path = './persons/content/persons/'
# files = os.listdir(directory_path)

# for file in files:
#       file_path = os.path.join(directory_path, file)
#       os.remove(file_path)
#       print(f"Deleted: {file_path}")
